In [1]:
import math
import random
import time
import tqdm
import submodlib
import faiss
import pickle
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from helper_fns import taylor_softmax_v1
import torch
import os
from scipy.special import softmax

/opt/conda/envs/ingenious/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X=np.load("representations.npy")
# X=np.load("wikitext-103-glove-embeddings.npy")
N=X.shape[0]

In [3]:
idx=np.random.randint(0, N, size=10000).tolist()

In [4]:
representations=X[idx]
# faiss.normalize_L2(representations)
M=representations.shape[0]

In [5]:
# dist_mat=euclidean_distances(representations)
# data_sijs=np.exp(-dist_mat/dist_mat.mean())
data_sijs=cosine_similarity(representations)

In [ ]:
# data_sijs[data_sijs<0]=0

In [ ]:
# for s in sorted(data_sijs[0].tolist(), reverse=True):
#     print(s)

1.000000238418579
0.8869812488555908
0.8811548948287964
0.8752744197845459
0.8696535229682922
0.8681506514549255
0.8665367364883423
0.8653126358985901
0.8647499084472656
0.8644682168960571
0.8636292219161987
0.8631048798561096
0.8630894422531128
0.8625043630599976
0.8620544672012329
0.8619768023490906
0.861472487449646
0.8614062070846558
0.8611115217208862
0.8608015775680542
0.860015869140625
0.8599416613578796
0.8591753244400024
0.8590319156646729
0.8585143685340881
0.8572308421134949
0.8570472002029419
0.8560057282447815
0.8553869724273682
0.8551443219184875
0.8545104265213013
0.8538034558296204
0.8531289100646973
0.8530869483947754
0.8528034090995789
0.8517687320709229
0.8510981798171997
0.8510283827781677
0.8509310483932495
0.8507553339004517
0.8506998419761658
0.8505851030349731
0.8501643538475037
0.8494745492935181
0.8494666814804077
0.8494162559509277
0.8493386507034302
0.8488087058067322
0.8487971425056458
0.8487603664398193
0.8486337065696716
0.8486168384552002
0.8477973341941

In [6]:
obj = submodlib.FacilityLocationFunction(n = representations.shape[0], separate_rep=False, mode = 'dense', sijs = data_sijs)

In [7]:
greedyList=obj.maximize(budget=M-1, optimizer="LazyGreedy", stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False, show_progress=True)

[||||||||||||||||||||]100% [Iteration 9999 of 9999]

In [8]:
subset_indices=[p[0] for p in greedyList]
gains=[p[1] for p in greedyList]
probs=taylor_softmax_v1(torch.from_numpy(np.array([gains]))).numpy()[0]

In [9]:
for ind in subset_indices:
    print(idx[ind])

864490
213221
192429
856401
996089
1049075
546594
853481
358187
15228
789551
82234
171224
186120
1041670
385074
979072
50931
1062005
1048004
529256
41984
551114
994977
188072
363658
962624
622296
472908
194287
320289
640973
429130
35244
1017842
581877
269688
22916
141364
86892
613055
868050
850290
85852
698473
78165
924631
258774
659752
126238
812059
286298
290862
352822
709567
816941
509359
1055661
34361
958439
59631
1033410
831321
271696
1007773
638890
180632
919277
786274
156924
812261
310331
927715
998729
2564
330029
991256
62601
682653
1039744
341343
132671
989424
941977
299893
553587
425928
848011
443241
409371
965931
758195
823496
672875
3273
780807
250920
665411
960041
579781
1031883
917042
1069277
661572
478500
643842
969119
893321
905412
759298
874208
983719
116140
117196
620546
588076
217144
952198
679157
673426
258159
630275
885728
659760
76780
991136
721872
649948
334102
325510
286784
261584
652339
68946
1019102
615894
877143
376842
450905
273889
46667
746750
19375
603027


In [10]:
for g in gains:
    print(g)

7548.841576993465
176.0662896335125
115.47182014584541
74.19435277581215
46.814407378435135
35.15595918893814
25.94881296157837
23.743327379226685
22.34349286556244
22.120033353567123
21.230553179979324
20.18696027994156
16.5569866001606
16.274998784065247
16.024261593818665
13.920353710651398
12.118914127349854
11.364643096923828
10.5535289645195
9.605657279491425
9.023431181907654
7.909106552600861
7.573080480098724
6.92502748966217
6.74552395939827
6.708610117435455
6.52638041973114
6.290546894073486
6.051536560058594
5.906038582324982
5.663875102996826
5.586491584777832
5.1758522391319275
5.15930300951004
4.98635521531105
4.804097652435303
4.336207389831543
4.270792543888092
4.103050649166107
4.065721273422241
4.055496424436569
3.9669179916381836
3.9481369256973267
3.9053930044174194
3.848903477191925
3.801403224468231
3.6321082711219788
3.580803871154785
3.4506660103797913
3.4247584342956543
3.1916378140449524
3.1526876091957092
3.137656092643738
3.041620135307312
2.98409700393676

In [11]:
for p in probs:
    print(p)

0.9985250294804571
0.0005492485210585005
0.00023766050107730148
9.906734504495037e-05
4.006736164296694e-05
2.2917916760772836e-05
1.2739727685925485e-05
1.0742571598398796e-05
9.563375515446332e-06
9.381491604966512e-06
8.674844716346195e-06
7.881102765945684e-06
5.4173869737858866e-06
5.2453222489990235e-06
5.094666128237652e-06
3.917310527786123e-06
3.0324605565590683e-06
2.6957391327095633e-06
2.3558844761944425e-06
1.9879363221529563e-06
1.7775318448710326e-06
1.4079557847627504e-06
1.3050471044824157e-06
1.1177513508753963e-06
1.0684747724527929e-06
1.0584812821593586e-06
1.0098467643732987e-06
9.486332992083359e-07
8.885834121798283e-07
8.530079785528056e-07
7.954415874848577e-07
7.774793829012324e-07
6.856727054193476e-07
6.820966382106371e-07
6.452990500885358e-07
6.076547422853726e-07
5.163434770590909e-07
5.041885555220781e-07
4.7370507907762725e-07
4.670553721585124e-07
4.6524247595561006e-07
4.496905700789197e-07
4.464284598492526e-07
4.3905027601144177e-07
4.2939761260216

In [12]:
idx2=idx.copy()
idx2.pop(subset_indices[0])
representations=X[idx2]
# faiss.normalize_L2(representations)
M=representations.shape[0]
M

9999

In [13]:
# dist_mat=euclidean_distances(representations)
# data_sijs=np.exp(-dist_mat/dist_mat.mean())
data_sijs=cosine_similarity(representations)

In [14]:
obj = submodlib.FacilityLocationFunction(n = representations.shape[0], separate_rep=False, mode = 'dense', sijs = data_sijs)

In [15]:
greedyList=obj.maximize(budget=M-1, optimizer="LazyGreedy", stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False, show_progress=True)

[||||||||||||||||||||]100% [Iteration 9998 of 9998]

In [16]:
subset_indices2=[p[0] for p in greedyList]
gains=[p[1] for p in greedyList]
probs=taylor_softmax_v1(torch.from_numpy(np.array([gains]))).numpy()[0]

In [17]:
for ind in subset_indices2:
    print(idx2[ind])

320289
213221
192429
856401
996089
491990
1049075
186120
853481
15228
358187
258774
478500
171224
1041670
979072
385074
168993
529256
1062005
1048004
363658
773818
551114
622296
188072
962624
924631
472908
194287
640973
1017842
919277
429130
290862
972726
581877
753199
958439
22916
141364
86892
48324
850290
874600
698473
78165
812059
868050
659752
62601
85852
352822
59631
286298
709567
82234
1055661
34361
1033410
202157
994977
816941
1007773
638890
180632
41984
786274
156924
509359
341343
998729
1039744
682653
2564
927715
989424
823496
132671
330029
965931
409371
443241
1019102
310331
553587
831321
299893
812261
271696
941977
848011
665411
425928
672875
661572
960041
116140
780807
579781
1031883
917042
250920
3273
719607
758195
969119
893321
1069277
759298
643842
983719
884555
679157
258159
620546
588076
952198
117196
630275
111931
649948
35244
960535
286784
76780
261584
991136
68946
258699
334102
652339
905412
325510
877143
283877
450905
46667
217144
94953
173761
603027
329734
904399


In [18]:
for g in gains:
    print(g)

7542.828100502491
139.76573964953423
133.06133341789246
82.8418088555336
43.43525767326355
41.15218111872673
31.586181491613388
24.01556122303009
23.743327379226685
22.375789672136307
21.738948106765747
21.24083387851715
16.094591796398163
15.94931423664093
15.500355303287506
14.880844950675964
13.702200025320053
11.569794058799744
9.803246796131134
9.604541063308716
9.251426011323929
8.535951733589172
7.091132819652557
7.034778356552124
6.74418705701828
6.709415704011917
6.514212131500244
6.482324481010437
6.337465524673462
5.906038582324982
5.632406294345856
5.3989078104496
5.092181444168091
5.088292837142944
5.0849427580833435
4.735700726509094
4.695513725280762
4.542482018470764
4.484708726406097
4.287364721298218
4.168534338474274
4.025591194629669
3.9820280373096466
3.942763030529022
3.892960846424103
3.8584506511688232
3.8369263410568237
3.654569447040558
3.5243014693260193
3.3607035279273987
3.343090832233429
3.3181850910186768
3.2138956785202026
3.1918108463287354
3.1566931605

In [19]:
for p in probs:
    print(p)

0.9986111441835963
0.00034771920945856114
0.0003153872022430326
0.00012336611550290859
3.466475507460721e-05
3.119588366283037e-05
1.8650418409213183e-05
1.099833104912969e-05
1.0760632998478592e-05
9.605931330665923e-06
9.09060304848982e-06
8.697452282045639e-06
5.145338248485674e-06
5.058552032507837e-06
4.795033427687635e-06
4.4430239603408394e-06
3.8105006321078984e-06
2.7900321015349808e-06
2.065507206381824e-06
1.9908631859867773e-06
1.861634360792236e-06
1.6132094726861504e-06
1.1663131928763568e-06
1.150366691554465e-06
1.0699078149091782e-06
1.060478846778997e-06
1.0083331165639929e-06
9.999418730072196e-07
9.622714727184948e-07
8.544421340742077e-07
7.894368057403857e-07
7.36043708841156e-07
6.688135318445615e-07
6.679823977677662e-07
6.672667918607747e-07
5.94826455486403e-07
5.867654205053012e-07
5.565879302237786e-07
5.45408881842882e-07
5.081065417546873e-07
4.863042718597602e-07
4.607345419727263e-07
4.530845152023653e-07
4.462463453636699e-07
4.3765091649128604e-07
4.31

In [20]:
from datasets import load_from_disk
from transformers import BertTokenizerFast

In [21]:
dataset=load_from_disk("wikitext-103-first-sentences-text")

In [22]:
dataset["train"][idx[subset_indices[0]]]["text"]

"before income is earned from tenants. the newspaper was also unhappy with complaints from tenants ; specific reasons for their dissatisfaction are never given. the age was disappointed by the lack of pre @ - @ built towers and scenarios, suggesting that one along the lines of the towering inferno's plot could have been included. game inform"

In [23]:
dataset["train"][idx[subset_indices2[0]]]["text"]

"on the material, as it constitutes an entire curriculum beyond the one provided by the school. around the turn of the millennium, several coaches protested the usad's decision to publish error @ - @ ridden resource guides rather than provide topics for students to research. = = history = ="

In [ ]:
# subsets=[]
# rng=np.random.default_rng(int(time.time()))
# for sample in range(10):
#     subsets.append(set(rng.choice(list(range(M-1)), size=int((M-1)*0.25), replace=False, p=probs).tolist()))

In [ ]:
# s=subsets[0]
# for i in range(1, 10):
#     print(len(s))
#     s=s.intersection(subsets[i])

2499
684
261
129
89
71
66
60
58


In [ ]:
subsets=[]
rng=np.random.default_rng(int(time.time()))
for sample in range(10):
    subsets.append(set(rng.choice(list(range(M-1)), size=int((M-1)*0.25), replace=False, p=probs).tolist()))

In [ ]:
s=subsets[0]
for i in range(1, 10):
    print(len(s))
    s=s.intersection(subsets[i])

6749
1832
575
242
164
135
121
117
114
